In [49]:
#Парсер комментариев и соцдем хараткеристик ВК (за последние 5 лет)
import configparser
import vk_api
from vk_api.exceptions import ApiError
import pandas as pd
from datetime import datetime, timedelta

def get_user_details(vk, user_ids):
    if not user_ids:
        return {}
    try:
        users = vk.users.get(user_ids=user_ids, fields='sex,bdate')
        return {user['id']: {
            'gender': 'мужской' if user['sex'] == 2 else 'женский' if user['sex'] == 1 else 'не указан',
            'birthdate': user.get('bdate', '')
        } for user in users}
    except ApiError as e:
        print(f"Ошибка при получении данных пользователя: {e}")
        return {}

def fetch_all_comments(vk, owner_id, post_id, end_time):
    all_comments = []
    offset = 0
    while True:
        try:
            comments_response = vk.wall.getComments(owner_id=owner_id, post_id=post_id, count=100, offset=offset, extended=1)
            comments = comments_response['items']
            if not comments:
                break
            for comment in comments:
                comment_date = datetime.fromtimestamp(comment['date'])
                if comment_date < end_time:
                    continue
                all_comments.append(comment)
            offset += 100
        except ApiError as e:
            print(f"Ошибка при загрузке комментариев: {e}")
            break
    return all_comments

def main():
    config = configparser.ConfigParser()
    config.read('config.ini')  #файл для выгрузки токена API VK
    token = config.get('VK', 'TOKEN')

    vk_session = vk_api.VkApi(token=token)
    vk = vk_session.get_api()

    keywords = ["экологический активизм", "экоактивизм", "экология", "эко"]
    rows = []
    closed_groups_count = 0
    
    five_years_ago = datetime.now() - timedelta(days=5 * 365)

    for keyword in keywords:
        try:
            groups = vk.groups.search(q=keyword, count=10, type='group', filter='publics')
        except ApiError as error:
            print(f"Ошибка поиска групп по ключевому слову {keyword}: {error}")
            continue
        
        for group in groups['items']:
            if group['is_closed']:
                closed_groups_count += 1
                continue
            
            group_id = group['id']
            try:
                group_info = vk.groups.getById(group_ids=group_id, fields='members_count')[0]
                group_name = group_info['name']
                group_members_count = group_info.get('members_count', 'недоступно')
                
                posts = vk.wall.get(owner_id=f'-{group_id}', count=100)
            except ApiError as e:
                print(f"Ошибка при получении информации о группе или постах из группы ID {group_id}: {e}")
                continue
            
            for post in posts['items']:
                post_id = post['id']
                post_date = datetime.fromtimestamp(post['date'])
                if post_date < five_years_ago:
                    continue
                
                comments = fetch_all_comments(vk, f'-{group_id}', post_id, five_years_ago)
                for comment in comments:
                    user_ids = [str(comment['from_id']) for comment in comments if comment['from_id'] > 0]
                    user_details = get_user_details(vk, user_ids)
                    
                    for comment in comments:
                        user_id = comment['from_id']
                        if user_id < 0:
                            continue
                        user_info = user_details.get(user_id, {})
                        likes_count = comment.get('likes', {}).get('count', 0) if 'likes' in comment else 0

                        rows.append({
                            'Название группы': group_name,
                            'ID группы': group_id,
                            'Количество участников': group_members_count,
                            'ID поста': post_id,
                            'Дата поста': post_date.strftime('%Y-%m-%d %H:%M:%S'),
                            'ID автора': user_id,
                            'Ссылка на автора': f'https://vk.com/id{user_id}',
                            'Пол автора': user_info.get('gender', ''),
                            'Возраст автора': user_info.get('birthdate', ''),
                            'Ссылка на комментарий': f'https://vk.com/wall-{group_id}_{post_id}?reply={comment["id"]}',
                            'Дата и время': datetime.fromtimestamp(comment['date']).strftime('%Y-%m-%d %H:%M:%S'),
                            'Текст комментария': comment['text'],
                            'Число лайков к комментарию': likes_count
                        })
                
    df = pd.DataFrame(rows)
    df.to_csv('your file name here.csv', index=False, encoding='utf_8_sig')
    print(f"Количество закрытых групп, с которых не удалось добыть данные: {closed_groups_count}")
    print(f"Сохранено {len(rows)} записей.")
    return df

if __name__ == '__main__':
    df = main()


Количество закрытых групп, с которых не удалось добыть данные: 3


In [ ]:
#Парсер постов и их хараткеристик ВК (за последние 5 лет)
import configparser
import vk_api
from vk_api.exceptions import ApiError
import pandas as pd
from datetime import datetime, timedelta

def main():
    config = configparser.ConfigParser()
    config.read('config.ini') #файл для выгрузки токена API VK
    token = config.get('VK', 'TOKEN')

    vk_session = vk_api.VkApi(token=token)
    vk = vk_session.get_api()

    keywords = ["экологический активизм", "экоактивизм", "экология", "эко"]
    rows = []
    closed_groups_count = 0
    five_years_ago = datetime.now() - timedelta(days=5 * 365)

    for keyword in keywords:
        try:
            groups = vk.groups.search(q=keyword, count=10, type='group', filter='publics')
        except ApiError as error:
            print(f"Ошибка поиска групп по ключевому слову {keyword}: {error}")
            continue
        
        for group in groups['items']:
            if group['is_closed']:
                closed_groups_count += 1
                continue
            
            group_id = group['id']
            try:
                group_info = vk.groups.getById(group_ids=group_id, fields='members_count')[0]
                group_name = group_info['name']
                group_members_count = group_info.get('members_count', 'недоступно')
                
                posts = vk.wall.get(owner_id=f'-{group_id}', count=100)
                for post in posts['items']:
                    post_date = datetime.fromtimestamp(post['date'])
                    if post_date < five_years_ago:
                        continue

                    rows.append({
                        'Название группы': group_name,
                        'ID группы': group_id,
                        'Количество участников': group_members_count,
                        'ID поста': post['id'],
                        'Дата поста': post_date.strftime('%Y-%m-%d %H:%M:%S'),
                        'Текст поста': post['text'],
                        'Количество лайков': post.get('likes', {}).get('count', 0),
                        'Количество комментариев': post.get('comments', {}).get('count', 0),
                        'Количество репостов': post.get('reposts', {}).get('count', 0),
                        'Количество просмотров': post.get('views', {}).get('count', 0)
                    })
            except ApiError as e:
                print(f"Ошибка при получении информации о группе или постах из группы ID {group_id}: {e}")

    df = pd.DataFrame(rows)
    df.to_csv('ypur file name here.csv', index=False, encoding='utf_8_sig')
    print(f"Количество закрытых групп, с которых не удалось добыть данные: {closed_groups_count}")
    print(f"Сохранено {len(rows)} записей.")
    return df

if __name__ == '__main__':
    df = main()


In [ ]:
#Парсер сайта Change.org (в разработке)
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def setup_driver():
    # Настройка драйвера Selenium для Chrome
    options = Service(ChromeDriverManager().install())
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--headless')  # Запуск без GUI, если не нужно визуально видеть браузер
    driver = webdriver.Chrome(executable_path='путь_к_вашему_chromedriver', options=options)
    return driver

def search_petitions(keyword):
    driver = setup_driver()
    driver.get("https://www.change.org/search")
    time.sleep(2)  # Подождите немного, пока страница загрузится

    try:
        # Ввод ключевого слова в поле поиска и отправка формы
        search_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, "q"))
        )
        search_input.send_keys(keyword)
        search_input.send_keys(Keys.RETURN)

        # Ожидание загрузки результатов поиска
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".search-result"))
        )

        # Получение результатов поиска
        results = driver.find_elements(By.CSS_SELECTOR, ".search-result")
        petitions = []
        for result in results:
            title = result.find_element(By.CSS_SELECTOR, 'a.search-result-title').text.strip()
            link = result.find_element(By.CSS_SELECTOR, 'a.search-result-title').get_attribute('href')
            description = result.find_element(By.CSS_SELECTOR, '.search-result-snippet').text.strip()
            creator = result.find_element(By.CSS_SELECTOR, '.search-result-byline').text.strip()

            petitions.append({
                'title': title,
                'link': link,
                'description': description,
                'creator': creator
            })

        # Вывод результатов
        for petition in petitions:
            print(petition)
    finally:
        driver.quit()

# Пример использования функции
search_petitions("environment")